In [1]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)
tf.test.is_gpu_available()

gpus = tf.config.list_physical_devices('GPU')
if gpus: 
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=14336 )] #7168  6144
    )

logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5511569699086511556
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2958553908
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15052127645038666387
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 960M, pci bus id: 0000:01:00.0, compute capability: 5.0"
xla_global_id: 416903419
]
Name: /physical_device:GPU:0   Type: GPU
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
1 Physical GPU, 1 Logical GPUs


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from utils import *
import json
import random
#from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish

from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import classification_report, accuracy_score, make_scorer
from keras.utils import np_utils

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline


In [7]:
#!pip install fasttext

     ---------------------------------------- 68.8/68.8 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-win_amd64.whl size=222525 sha256=becc77c904f43ccc7bc5a6474e297542e5fbc307984d8972b3fa42eb2d022caa
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\3f\84\86\c63cf501c46fb575152daee4b937075a5e9b31765c0e620fd4
Successfully built fasttext


In [3]:
column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_total.xlsx")

#column = ['tweets','duygu']
#df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")
#df = pd.read_excel("../dataset/preprocessing_kemik.xlsx")

df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [4]:
df=df.sample(frac=1).reset_index(drop=True)
df

,tweets,duygu
0,whatsapp'tan vazgecelim. yazilimi ve serverlar...,2
1,"ay deli gibi oldum yemin ederim, turkcell'in r...",2
2,başlarım turkcell’e de 3g’sine de. beceriksizl...,1
3,turkcell fakirin halinden anlıyo açık net,2
4,ahasjhdaşdhasd:akp-turkcell chp-avea ve mhp-vo...,0
...,...,...
17246,turkcell'e lira yüklüyorum 15 tane reklam mesa...,1
17247,#turkcellkullanmiyorumcunku bi saniye lan ben ...,0
17248,5 gb hediye internet veren turkcell ne kadar d...,2
17249,turkcell soydun yine beni !!! her ay olmaz böyle,1


In [5]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [8]:
# Load my word embeddings
#import gensim
#wordembeddings = gensim.models.KeyedVectors.load_word2vec_format('drive/My Drive/CNNPhraseEmbeddings/Resources/GoogleNews-vectors-negative300.bin', binary=True)

import fasttext
import fasttext.util
wordembeddings = fasttext.load_model('D:/Desktop/İndirilenler yedek/cc.tr.300.bin')

In [9]:
def fasttext (X_train,y_train,X_test,y_test):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)
    sequences_X_train = tokenizer.texts_to_sequences(X_train)
    word_index = tokenizer.word_index
    max_length = 0
    for review_number in range(len(sequences_X_train)):
        numberofwords=len(sequences_X_train[review_number])
        if (numberofwords) > (max_length):
            max_length = numberofwords

    X_train = pad_sequences(sequences_X_train, maxlen=max_length)
    y_train = np.asarray(y_train)
    y_train = np_utils.to_categorical(y_train)

    sequences_X_test = tokenizer.texts_to_sequences(X_test)
    X_test = pad_sequences(sequences_X_test, maxlen=max_length)
    y_test = np.asarray(y_test)
    y_test = np_utils.to_categorical(y_test)

    unique_words = len(word_index)
    total_words = unique_words + 1
    skipped_words = 0
    embedding_dim = 300
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items():
        try:
            embedding_vector = wordembeddings[word]
        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
      
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)

    return embedding_layer,X_train,y_train,X_test,y_test

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1,mcc,cohen_kappa=[],[],[],[],[],[]
pat = 3
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
 
    embedding_layer,X_train,y_train,X_test,y_test= fasttext (X_train,y_train,X_test,y_test)
 
    model = Sequential() #rnn
    model.add(embedding_layer)
    model.add(SimpleRNN(256,activation='relu',return_sequences= True))
    model.add(SimpleRNN(128,activation='relu'))
    model.add(Dense(3, activation='softmax'))

    opt = tf.keras.optimizers.SGD(learning_rate=0.05)
    model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy']) #binary cross çünkü sonucun pozitif yada negatif


    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    model.fit(X_train, y_train, epochs=75,callbacks=early_stopping)

    # evaluate
    loss, accuracy = model.evaluate(X_test, y_test)
    preds = model.predict(X_test)
    y_true=y_test.argmax(axis=1)
    y_pred=preds.argmax(axis=1)
    
    precision= precision_score(y_true, y_pred, average='weighted')
    recall= recall_score(y_true, y_pred, average='weighted')
    f1_measure = f1_score(y_true, y_pred, average='weighted')
    mcc_score = matthews_corrcoef(y_true, y_pred)
    c_kappa=cohen_kappa_score(y_true, y_pred)
    
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_measure)
    mcc.append(mcc_score)
    cohen_kappa.append(c_kappa)
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/75
486/486 [==============================] - 212s 433ms/step - loss: 1.0212 - accuracy: 0.4814
Epoch 2/75
486/486 [==============================] - 212s 435ms/step - loss: 0.9250 - accuracy: 0.5721
Epoch 3/75
486/486 [==============================] - 212s 436ms/step - loss: nan - accuracy: 0.5476
Epoch 4/75
486/486 [==============================] - 211s 434ms/step - loss: nan - accuracy: 0.3362
Epoch 5/75
486/486 [==============================] - 213s 437ms/step - loss: nan - accuracy: 0.3362
Epoch 5: early stopping
54/54 [==============================] - 2s 35ms/step


D:\Program\Anaconda\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/75
486/486 [==============================] - 204s 417ms/step - loss: 1.0299 - accuracy: 0.4844
Epoch 2/75
486/486 [==============================] - 202s 416ms/step - loss: 0.9393 - accuracy: 0.5659
Epoch 3/75
486/486 [==============================] - 197s 406ms/step - loss: 0.8874 - accuracy: 0.6025
Epoch 4/75
486/486 [==============================] - 197s 405ms/step - loss: 0.8533 - accuracy: 0.6167
Epoch 5/75
486/486 [==============================] - 198s 408ms/step - loss: 0.8239 - accuracy: 0.6328
Epoch 6/75
486/486 [==============================] - 198s 407ms/step - loss: 0.7962 - accuracy: 0.6512
Epoch 7/75
486/486 [==============================] - 199s 410ms/step - loss: 0.7592 - accuracy: 0.6704
Epoch 8/75
486/486 [==============================] - 199s 410ms/step - loss: 0.7359 - accuracy: 0.6818
Epoch 9/75
486/486 [==============================] - 204s 420ms/step -

D:\Program\Anaconda\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------------------------------------------------------------------------
Training for fold 4 ...
Epoch 1/75
486/486 [==============================] - 203s 414ms/step - loss: 1.0259 - accuracy: 0.4805
Epoch 2/75
486/486 [==============================] - 201s 414ms/step - loss: 0.9339 - accuracy: 0.5691
Epoch 3/75
486/486 [==============================] - 202s 416ms/step - loss: 0.8798 - accuracy: 0.6027
Epoch 4/75
486/486 [==============================] - 202s 415ms/step - loss: 0.8399 - accuracy: 0.6288
Epoch 5/75
486/486 [==============================] - 201s 414ms/step - loss: 0.8115 - accuracy: 0.6432
Epoch 6/75
486/486 [==============================] - 203s 417ms/step - loss: 0.7754 - accuracy: 0.6604
Epoch 7/75
486/486 [==============================] - 200s 412ms/step - loss: 0.7700 - accuracy: 0.6673
Epoch 8/75
486/486 [==============================] - 201s 413ms/step - loss: 0.7604 - accuracy: 0.6720
Epoch 9/75
486/486 [==============================] - 202s 415ms/step -

...

------------------------------------------------------------------------
Training for fold 5 ...
Epoch 1/75
486/486 [==============================] - 212s 434ms/step - loss: 1.0298 - accuracy: 0.4779
Epoch 2/75
486/486 [==============================] - 212s 436ms/step - loss: 0.9416 - accuracy: 0.5616
Epoch 3/75
486/486 [==============================] - 212s 436ms/step - loss: 0.8851 - accuracy: 0.5968
Epoch 4/75
486/486 [==============================] - 211s 435ms/step - loss: 0.8562 - accuracy: 0.6196
Epoch 5/75
486/486 [==============================] - 220s 452ms/step - loss: nan - accuracy: 0.3501
Epoch 6/75
486/486 [==============================] - 215s 443ms/step - loss: nan - accuracy: 0.3399
Epoch 7/75
486/486 [==============================] - 216s 445ms/step - loss: nan - accuracy: 0.3399
Epoch 7: early stopping
54/54 [==============================] - 2s 37ms/step


D:\Program\Anaconda\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------------------------------------------------------------------------
Training for fold 6 ...
Epoch 1/75
486/486 [==============================] - 213s 435ms/step - loss: 1.0235 - accuracy: 0.4857
Epoch 2/75
486/486 [==============================] - 210s 433ms/step - loss: 0.9394 - accuracy: 0.5664
Epoch 3/75
486/486 [==============================] - 210s 433ms/step - loss: 0.8821 - accuracy: 0.6054
Epoch 4/75
486/486 [==============================] - 212s 436ms/step - loss: 0.8480 - accuracy: 0.6221
Epoch 5/75
486/486 [==============================] - 210s 433ms/step - loss: 0.8216 - accuracy: 0.6365
Epoch 6/75
486/486 [==============================] - 213s 438ms/step - loss: 0.7842 - accuracy: 0.6610
Epoch 7/75
486/486 [==============================] - 206s 425ms/step - loss: 0.7614 - accuracy: 0.6703
Epoch 8/75
486/486 [==============================] - 206s 423ms/step - loss: 0.7305 - accuracy: 0.6853
Epoch 9/75
486/486 [==============================] - 204s 421ms/step -

D:\Program\Anaconda\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


------------------------------------------------------------------------
Training for fold 7 ...
Epoch 1/75
486/486 [==============================] - 204s 416ms/step - loss: 1.0293 - accuracy: 0.4784
Epoch 2/75
486/486 [==============================] - 202s 416ms/step - loss: 0.9536 - accuracy: 0.5539
Epoch 3/75
486/486 [==============================] - 198s 407ms/step - loss: 0.8903 - accuracy: 0.5976
Epoch 4/75
368/486 [=====================>........] - ETA: 50s - loss: 0.8522 - accuracy: 0.6231

In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

In [ ]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))
print('test mcc: %f' % (Average(mcc)))
print('test cohen_kappa: %f' % (Average(cohen_kappa)))

In [ ]:
#cnn

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1,mcc,cohen_kappa=[],[],[],[],[],[]
pat = 3
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
    
for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train]
    y_train=targets[train]
    X_test=inputs[test]
    y_test=targets[test]

    embedding_layer,X_train,y_train,X_test,y_test= fasttext (X_train,y_train,X_test,y_test)
    
    modelCNN = Sequential() #cnn

    modelCNN.add(embedding_layer)
    modelCNN.add(Conv1D(filters=128, kernel_size=3, activation='relu')) #kernal size 5 yan yana beş kelimeye bakması
    modelCNN.add(MaxPooling1D(pool_size=3)) #tek satırlık 1d olduğu için
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(64, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(16, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Flatten()) #düzgünleştirmek için
    modelCNN.add(Dense(3, activation='softmax'))
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.05)
    modelCNN.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy']) #binary cross çünkü sonucun pozitif yada negatif
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNN.fit(X_train, y_train, epochs=75,callbacks=early_stopping)

    # evaluate
    loss, accuracy = modelCNN.evaluate(X_test, y_test)
    preds = modelCNN.predict(X_test)
    y_true=y_test.argmax(axis=1)
    y_pred=preds.argmax(axis=1)
    
    precision= precision_score(y_true, y_pred, average='weighted')
    recall= recall_score(y_true, y_pred, average='weighted')
    f1_measure = f1_score(y_true, y_pred, average='weighted')
    mcc_score = matthews_corrcoef(y_true, y_pred)
    c_kappa=cohen_kappa_score(y_true, y_pred)
    
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_measure)
    mcc.append(mcc_score)
    cohen_kappa.append(c_kappa)
    
    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))
print('test mcc: %f' % (Average(mcc)))
print('test cohen_kappa: %f' % (Average(cohen_kappa)))

In [ ]:
#lstm

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1,mcc,cohen_kappa=[],[],[],[],[],[]
pat = 3
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
    
for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train]
    y_train=targets[train]
    X_test=inputs[test]
    y_test=targets[test]

    embedding_layer,X_train,y_train,X_test,y_test= fasttext (X_train,y_train,X_test,y_test)

    modelLSTM = Sequential()
    modelLSTM.add(embedding_layer)
    modelLSTM.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    modelLSTM.add(Dense(3, activation='softmax'))
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.05)
    modelLSTM.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy']) #binary cross çünkü sonucun pozitif yada negatif
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelLSTM.fit(X_train, y_train, epochs=75,callbacks=early_stopping)

    # evaluate
    loss, accuracy = modelLSTM.evaluate(X_test, y_test)
    preds = modelLSTM.predict(X_test)
    y_true=y_test.argmax(axis=1)
    y_pred=preds.argmax(axis=1)
    
    precision= precision_score(y_true, y_pred, average='weighted')
    recall= recall_score(y_true, y_pred, average='weighted')
    f1_measure = f1_score(y_true, y_pred, average='weighted')
    mcc_score = matthews_corrcoef(y_true, y_pred)
    c_kappa=cohen_kappa_score(y_true, y_pred)
    
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_measure)
    mcc.append(mcc_score)
    cohen_kappa.append(c_kappa)
    
    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))
print('test mcc: %f' % (Average(mcc)))
print('test cohen_kappa: %f' % (Average(cohen_kappa)))

In [ ]:
#cnn+lstm

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1,mcc,cohen_kappa=[],[],[],[],[],[]
pat = 3
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
    
for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train]
    y_train=targets[train]
    X_test=inputs[test]
    y_test=targets[test]

    embedding_layer,X_train,y_train,X_test,y_test= fasttext (X_train,y_train,X_test,y_test)

    modelCNNLSTM = Sequential() #cnn

    modelCNNLSTM.add(embedding_layer)
    modelCNNLSTM.add(Conv1D(filters=128, kernel_size=3, activation='relu')) #kernal size 5 yan yana beş kelimeye bakması
    modelCNNLSTM.add(MaxPooling1D(pool_size=3)) #tek satırlık 1d olduğu için
    modelCNNLSTM.add(Dropout(0.3))
    modelCNNLSTM.add(Dense(64, activation='relu'))
    modelCNNLSTM.add(Dropout(0.3))
    modelCNNLSTM.add(Dense(16, activation='relu'))
    modelCNNLSTM.add(Dropout(0.3))
    modelCNNLSTM.add(LSTM(128))
    modelCNNLSTM.add(Flatten()) #düzgünleştirmek için
    modelCNNLSTM.add(Dense(3, activation='softmax'))

    opt = tf.keras.optimizers.SGD(learning_rate=0.05)
    modelCNNLSTM.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy']) #binary cross çünkü sonucun pozitif yada negatif
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNNLSTM.fit(X_train, y_train, epochs=75,callbacks=early_stopping)

    # evaluate
    loss, accuracy = modelCNNLSTM.evaluate(X_test, y_test)
    preds = modelCNNLSTM.predict(X_test)
    y_true=y_test.argmax(axis=1)
    y_pred=preds.argmax(axis=1)
    
    precision= precision_score(y_true, y_pred, average='weighted')
    recall= recall_score(y_true, y_pred, average='weighted')
    f1_measure = f1_score(y_true, y_pred, average='weighted')
    mcc_score = matthews_corrcoef(y_true, y_pred)
    c_kappa=cohen_kappa_score(y_true, y_pred)
    
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_measure)
    mcc.append(mcc_score)
    cohen_kappa.append(c_kappa)
    
    # Increase fold number
    fold_no = fold_no + 1

In [ ]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))
print('test mcc: %f' % (Average(mcc)))
print('test cohen_kappa: %f' % (Average(cohen_kappa)))